In [15]:
import os
import numpy as np
import cv2
import tensorflow as tf

In [16]:
IMG_SIZE = (224, 224)

In [17]:
def load_dataset(data_dir):
    images = []
    labels = []
    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                img_path = os.path.join(folder_path, filename)
                # 读取图像
                img = cv2.imread(img_path)
                # 调整图像大小
                img = cv2.resize(img, IMG_SIZE)
                # 将图像添加到列表中
                images.append(img)
                # 添加标签
                labels.append(folder)
    # 将标签转换为数字
    label_to_id = {'O': 0, 'R': 1}
    labels = [label_to_id[label] for label in labels]
    return np.array(images), np.array(labels)

In [18]:
train_images, train_labels = load_dataset('C:/Users/35468/data/test')
test_images, test_labels = load_dataset('C:/Users/35468/data/train')

In [19]:
train_images = train_images / 255.0
test_images = test_images /255.0

In [20]:
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [21]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [22]:
def create_resnet50v2_model():
    base_model = ResNet50V2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
resnet50v2_model = create_resnet50v2_model()

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [25]:
def create_cnn_model(input_shape=(224, 224, 3)):
    # 创建一个 Sequential 模型
    model = Sequential([
        # 添加卷积层
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2,2)),
        # 添加卷积层
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        # 添加卷积层
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        # 添加 Flatten 层
        Flatten(),
        # 添加全连接层
        Dense(64, activation='relu'),
        Dropout(0.5),
        # 添加输出层
        Dense(2, activation='softmax')
    ])
    # 编译模型
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [26]:
cnn_model = create_cnn_model()

In [27]:
history_resnet50v2 = resnet50v2_model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Epoch 1/5
10/10 [==============================] - 114s 12s/step - loss: 0.2983 - accuracy: 0.9100 - val_loss: 1.3130 - val_accuracy: 0.5635
Epoch 2/5
10/10 [==============================] - 106s 12s/step - loss: 0.0386 - accuracy: 0.9933 - val_loss: 2.1406 - val_accuracy: 0.5412
Epoch 3/5
10/10 [==============================] - 107s 12s/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 2.5808 - val_accuracy: 0.5365
Epoch 4/5
10/10 [==============================] - 110s 12s/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 2.8319 - val_accuracy: 0.5353
Epoch 5/5
10/10 [==============================] - 110s 12s/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 3.1806 - val_accuracy: 0.5271


In [28]:
history = cnn_model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Epoch 1/5
10/10 [==============================] - 28s 3s/step - loss: 1.0014 - accuracy: 0.6800 - val_loss: 0.4436 - val_accuracy: 0.7976
Epoch 2/5
10/10 [==============================] - 26s 3s/step - loss: 0.4266 - accuracy: 0.8867 - val_loss: 0.4740 - val_accuracy: 0.8424
Epoch 3/5
10/10 [==============================] - 26s 3s/step - loss: 0.2121 - accuracy: 0.9233 - val_loss: 0.4597 - val_accuracy: 0.8206
Epoch 4/5
10/10 [==============================] - 26s 3s/step - loss: 0.1522 - accuracy: 0.9633 - val_loss: 0.4672 - val_accuracy: 0.8406
Epoch 5/5
10/10 [==============================] - 26s 3s/step - loss: 0.1642 - accuracy: 0.9433 - val_loss: 0.5344 - val_accuracy: 0.8153


In [29]:
resnet50v2_model.evaluate(test_images, test_labels)

54/54 [==============================] - 94s 2s/step - loss: 3.1806 - accuracy: 0.5271


[3.1805543899536133, 0.5270588397979736]

In [30]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_images, test_labels)
print("CNN 模型的准确率: {:.2f}%".format(cnn_accuracy * 100))

54/54 [==============================] - 17s 311ms/step - loss: 0.5344 - accuracy: 0.8153
CNN 模型的准确率: 81.53%


In [31]:
import pickle

TypeError: unsupported operand type(s) for +: 'Sequential' and 'Sequential'

In [ ]:
with open('laji.pkl','wb') as f:
    pickle.dump(cnn_model,f)